# Import Library & autentikasi

In [ ]:
#!pip install earthengine-api

# Import the Earth Engine API and authenticate
import ee
ee.Authenticate()
ee.Initialize(project='ee-analafeanalber')

# Read Raster

In [ ]:
shapefile = ee.FeatureCollection('projects/ee-analafeanalber/assets/SHPLamsel')
image_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(shapefile) \
    .filterDate('2021-01-01', '2021-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))  #2021-01-01 2021-12-31
image = image_collection.median().select(['B4', 'B3', 'B2'])  # Red, Green, Blue


# NDBI

In [ ]:
image_collection = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(shapefile) \
    .filterDate('2021-01-01', '2021-12-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) #2021-01-01 2021-12-31

# Select the relevant bands: SWIR1 and NIR for NDBI calculation
image = image_collection.median()

# Calculate NDBI: (SWIR - NIR) / (SWIR + NIR)
ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')

# Threshold the NDBI to create a binary mask (urban vs non-urban)
urban_mask = ndbi.gt(0)

# Apply the mask to get urban areas
urban_image = urban_mask.updateMask(urban_mask)


# Output NDBI

In [ ]:
# Calculate the number of connected components (buildings)
connected_components = urban_image.connectedComponents(ee.Kernel.plus(1), 128)
building_count = connected_components.select('labels').connectedPixelCount(maxSize=128)

# Reduce to the SHP boundary
building_count_per_village = building_count.reduceRegions(
    collection=shapefile,
    reducer=ee.Reducer.count(),
    scale=10
)

# Get the result
result = building_count_per_village.getInfo()

# Display the result
for feature in result['features']:
    village_name = feature['properties']['NAMOBJ']
    count = feature['properties']['count']
    print(f'Village: {village_name}, Number of Buildings: {count}')


Village: Sri Katon, Number of Buildings: 8824
Village: Sinar Ogan, Number of Buildings: 3350
Village: Sukanegara, Number of Buildings: 15420
Village: Sindang Sari, Number of Buildings: 19061
Village: Rejomulyo, Number of Buildings: 12112
Village: Serdang, Number of Buildings: 24557
Village: Sabah Balau, Number of Buildings: 17623
Village: Trimulyo, Number of Buildings: 10418
Village: Way Galih, Number of Buildings: 7631
Village: Galih Lunik, Number of Buildings: 14251
Village: Budi Lestari, Number of Buildings: 7226
Village: Lematang, Number of Buildings: 10874
Village: Jati Baru, Number of Buildings: 12462
Village: Jati Indah, Number of Buildings: 7971
Village: Kaliasin, Number of Buildings: 6669
Village: Purwodadi Simpang, Number of Buildings: 10170
Village: Babulang, Number of Buildings: 20
Village: Agom, Number of Buildings: 6217
Village: Negeri Pandan, Number of Buildings: 1858
Village: Munjuk Sampurna, Number of Buildings: 7135
Village: Margacatur, Number of Buildings: 14061
Vill

# Output in Dataframe

In [ ]:
import pandas as pd

# Calculate the number of connected components (buildings)
connected_components = urban_image.connectedComponents(ee.Kernel.plus(1), 128)
building_count = connected_components.select('labels').connectedPixelCount(maxSize=128)

# Reduce to the SHP boundary
building_count_per_village = building_count.reduceRegions(
    collection=shapefile,
    reducer=ee.Reducer.count(),
    scale=10
)

# Get the result
result = building_count_per_village.getInfo()

# Prepare lists to store village names and building counts
village_names = []
building_counts = []

# Loop through the results and populate the lists
for feature in result['features']:
    village_name = feature['properties']['NAMOBJ']  # Adjust 'NAMOBJ' according to your SHP's field name
    count = feature['properties']['count']
    village_names.append(village_name)
    building_counts.append(count)

# Create a DataFrame from the lists
df = pd.DataFrame({
    'desa': village_names,
    'jumlah bangunan': building_counts
})

df


,desa,jumlah bangunan
0,Sri Katon,8824
1,Sinar Ogan,3350
2,Sukanegara,15420
3,Sindang Sari,19061
4,Rejomulyo,12112
...,...,...
251,Rulung Mulya,7857
252,Candi Mas,10911
253,Branti Raya,16327
254,Mandah,13846


# Export to CSV

In [ ]:
df.to_csv('jumlahbangunanperdesaLamsel.csv', index=False)